<a href="https://colab.research.google.com/github/ivan74rus/MTS-ML-CUP/blob/v2/MTS_CUP_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##import

In [1]:
#подключить гугл диск
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#instal catboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 8.8 MB/s eta 0:00:00


In [1]:
import pandas as pd
import joblib
import numpy as np
import time
import pyarrow.parquet as pq
import scipy
import bisect
import sklearn.metrics as metrics
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

In [2]:
# константы
LOCAL_DATA_PATH = '/content/drive/MyDrive/DS/MTS/'
#SPLIT_SEED = 42
DATA_FILE = 'competition_data_final_pqt'
TIM_LIST = 'top_300urls_timur.xlsx'
TARGET_FILE = 'public_train.pqt'
SUBMISSION_FILE = 'submit_2.pqt'


# 1.Загружаем в пандус кусок  и готовим для ML.
 Так как ОЗУ всего 12 Гигов и не хватает, то будем выдергивать нужные фичи по кускам, сохранять все на диск в csv и затем с диска снова подгружать.    

  

### 1.0. Условие на нужные урлы 
 
Оставим только урлы из списка Тимура,   
Подгрузим submit    


In [ ]:
#оставляем урлы Тимура , это отсортированные топ 300 урл по реквестам, и руками выкинуты технические
timur_url_list_1 = pd.read_excel(f'{LOCAL_DATA_PATH}{TIM_LIST}')
#технические для дропа
timur_url_list_tech = timur_url_list_1[timur_url_list_1['url_host'] == 0]['Unnamed: 0'].values

#дропаем технические
timur_url_list = pd.read_csv(f'{LOCAL_DATA_PATH}top_all_urls.csv')
timur_url_list = timur_url_list.query('index not in @timur_url_list_tech')

# 3300 уже не хватает ОЗУ чтобы сшить, нужно разделять похоже
timur_url_list = timur_url_list.reset_index(drop=True).loc[:2500,'index'].values
len(timur_url_list)
#timur_url_list

2501

In [ ]:
#сабмиты и таргеты
#сразу дропаем ненужный возраст из таргет
id_for_target = pq.read_table(f'{LOCAL_DATA_PATH}/{TARGET_FILE}').to_pandas().query('age >=19 ') 


id_for_submit = pq.read_table(f'{LOCAL_DATA_PATH}/{SUBMISSION_FILE}').to_pandas() 

In [ ]:
#функция разбивки возраста по категориям
def age_bucket(x):
    return bisect.bisect_left([19,25,35,45,55,65], x)

#my 
def age_bascket(x):
  if x <= 25 :
    return 1
  elif 25 < x <= 35:
    return 2
  elif 35 < x <= 45:
    return 3
  elif 45 < x <= 55:
    return 4
  elif 55 < x <= 65:
    return 5
  return 6  
#bisect.bisect_right([19,26,36,46,56,66], x) # вроде так надо пишут

In [ ]:
age_bascket(33)

2

## 1.1 Загружаем из паркета ЦИКЛОМ нужные фичи и обрабатываем кусок.
Формируем часть дф готового к обработке, сохраняем на диск уже раздельно трайн и фо_сабмит выбокри. Все дял экономии памяти ОЗУ. Все разбивки  и сохраннеия все в цикле. Иначе финальный кусок не оббработать никак.

In [ ]:
part_list  = ['part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00001-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00002-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00003-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00004-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00005-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00006-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00007-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00008-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
              'part-00009-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
    
]

In [ ]:
# цикл грузит из паркета нужные даные и отсеивает по топ урлам из списка
for i in  range(0,10): #(len(part_list))
  
  #тут выбираем какие колонки из паркета грузим, можно через переменну определить
  # 'cpe_model_name' 'cpe_manufacturer_name',
  df = pq.read_table(f'{LOCAL_DATA_PATH}{part_list[i]}').select([	'cpe_model_name',	 'url_host',		'request_cnt',	'part_of_day','user_id']).to_pandas()
  
  #контроль цикла, загрузка из паркета в память.
  print ('part', i, 'load, shape = ',df.shape)
  # для подсчета соко пропало user_id после обрезки по урлам
  user_id_count = df['user_id'].nunique()
  print ('part', i, 'до обрезки user_id = ', df['user_id'].nunique())

  # дф по времени суток / нужно это делать на всем дф, со всеми урлами
  df_user_part_of_day = df.pivot_table(index='user_id', columns='part_of_day', values='request_cnt',aggfunc='count').reset_index()

  #обрезаем по урлам
  df = df.query('url_host in @timur_url_list')
  print ('part', i, 'load, shape после обрезки урлов = ',df.shape)
  print ('part', i, 'после обрезки user_id уменьшилось на :',  user_id_count - df['user_id'].nunique())
  # дф по урлам формируем 
  df_user_url = df.pivot_table(index='user_id', columns='url_host', values='request_cnt',aggfunc='count').reset_index()

  #обезаем дф до состояния каждый юзер 1 строчка
  df.drop(columns=['url_host', 'request_cnt', 'part_of_day'], inplace=True)
  df.drop_duplicates(inplace=True)
  
 
  #обьединяем все куски тк что снала идет полная по user_id таблица с part_of_day
  # ее объединяем how='outer', чтобы все строки сохранились
  # для строк где нет урлов,будет прогнозирвоаться по part_day и модели балалайки
  print ('df_user_part_of_day.shape',df_user_part_of_day.shape )
  df = df_user_part_of_day.merge(df, how='outer', on='user_id')
  print ('после "df = df_user_part_of_day.merge(df_',df.shape)
  df = df.merge(df_user_url, how='outer', on='user_id')
  print ('после всех merge размерность df',df.shape)
  
  #теперь разделяем на train submit 
  df_train = df.merge(id_for_target,  on='user_id')
  df_submit = df.merge(id_for_submit, on='user_id')

  # кэт ругается на нан
  df_train.loc[df_train['cpe_model_name'].isna() ,'cpe_model_name'] = '0'

  #дропаем возраст <19  сразу тут из  трайн (убирается в merge)
  #df_train = df_train[df_train['age'] >=19] 
  
  #категоризируем возраст - заменна функция на мою
  df_train['age'] = df_train['age'].map(age_bascket)
  
  #сохраняем на диск  подготовленные файлы
  
  #df_train.reset_index(drop=True).to_pickle(f'{LOCAL_DATA_PATH}train_part_{i}.pkl')
  df_submit.to_feather(f'{LOCAL_DATA_PATH}submit_part_{i}.feather')

  # все заранее разбиваем, т к потом не хватает ОЗУ общий файл обрабатывать
  
  # сохраняем фичи и таргет для age
  df_train.drop(['user_id', 'age', 'is_male'], axis = 1).reset_index(drop=True).to_feather(f'{LOCAL_DATA_PATH}train_feat_age_part_{i}.feather') #,index = False)
  #df_train.drop(['user_id', 'age', 'is_male'], axis = 1).to_csv(f'{LOCAL_DATA_PATH}train_feat_age_part_{i}.csv' ,index = False)
  df_train['age'].to_csv(f'{LOCAL_DATA_PATH}train_target_age_part_{i}.csv', index = False)
    

  # сохраняем фичи и таргет для is_male
  #дропаем перед ML по полу, заранее нельзя, тк  там много данных по возрасту
  print ('df_train.shape до обрезки is_male != NA',df_train.shape )
  df_train = df_train[df_train['is_male'] != 'NA']
  print ('df_train.shape ПОСЛЕ обрезки is_male != NA',df_train.shape )
  
  df_train.drop(['user_id', 'age', 'is_male'], axis = 1).reset_index(drop=True).to_feather(f'{LOCAL_DATA_PATH}train_feat_is_male_part_{i}.feather')
  df_train['is_male'].to_csv(f'{LOCAL_DATA_PATH}train_target_is_male_part_{i}.csv', index = False)


  #итого сохраняется файл сабмита + общий_трайн + фич+таргет_age + фич+таргет_is_male = 6шт
  # все нужно для анализа

  print ('part' , i , 'saved on disc')

part 0 load, shape =  (32638709, 5)
part 0 до обрезки user_id =  41594
part 0 load, shape после обрезки урлов =  (15204490, 5)
part 0 после обрезки user_id уменьшилось на : 419
df_user_part_of_day.shape (41594, 5)
после "df = df_user_part_of_day.merge(df_ (41594, 6)
после всех merge размерность df (41594, 2506)
df_train.shape до обрезки is_male != NA (27081, 2508)
df_train.shape ПОСЛЕ обрезки is_male != NA (26529, 2508)
part 0 saved on disc
part 1 load, shape =  (32423119, 5)
part 1 до обрезки user_id =  41570
part 1 load, shape после обрезки урлов =  (15048716, 5)
part 1 после обрезки user_id уменьшилось на : 497
df_user_part_of_day.shape (41570, 5)
после "df = df_user_part_of_day.merge(df_ (41570, 6)
после всех merge размерность df (41570, 2506)
df_train.shape до обрезки is_male != NA (26936, 2508)
df_train.shape ПОСЛЕ обрезки is_male != NA (26394, 2508)
part 1 saved on disc
part 2 load, shape =  (32146620, 5)
part 2 до обрезки user_id =  41335
part 2 load, shape после обрезки урлов 

In [ ]:
print (df_user_part_of_day.shape )
print (df.shape)
print (df_submit.shape)

(41594, 5)
(41175, 2411)
(14237, 2411)


# 2.Склейка и сохранение  итогового файла. 
Тут уже файлы готовые , относительно не большие. сшиваются и сохр на диск,   хватает ОЗУ. 

In [54]:
list_train  = [
    'train_part_0.feather',
    'train_part_1.feather',
    'train_part_2.feather',
    'train_part_3.feather',
    'train_part_4.feather',
    'train_part_5.feather',
    'train_part_6.feather',
    'train_part_7.feather',
    'train_part_8.feather',
    'train_part_9.feather',
     ]

list_submit  = [
    'submit_part_0.feather',
    'submit_part_1.feather',
    'submit_part_2.feather',
    'submit_part_3.feather',
    'submit_part_4.feather',
    'submit_part_5.feather',
    'submit_part_6.feather',
    'submit_part_7.feather',
    'submit_part_8.feather',
    'submit_part_9.feather',
     ]


In [ ]:
%%time
#цикл склейки train общего
#первый файл
df = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train[0]}')
print (list_train[0],'shape', df.shape)


for i in range (1,len(list_train)):
  df_ = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train[i]}')
       
  #сшиваем
  df = pd.concat([df, df_], ignore_index=True)
  #контролль цикла
  print (list_train[i],'shape', df_.shape)
  print ('Суммарное df.shape',df.shape)


# сохранение

#df.to_csv(f'{LOCAL_DATA_PATH}/train_df_all_parts.csv')
#df.to_pickle(f'{LOCAL_DATA_PATH}/train_df_all_parts.pkl')
df.to_feather(f'{LOCAL_DATA_PATH}/train_df_all_parts.feather')
print ('Итоговый файл сохранен на диск')

FileNotFoundError: ignored

### цикл склейки submit

In [56]:
#цикл склейки submit
#первый файл
df = pd.read_feather(f'{LOCAL_DATA_PATH}{list_submit[0]}')
df.loc[df['cpe_model_name'].isna() ,'cpe_model_name'] = '0'
print (list_submit[0],'shape', df.shape)

for i in range (1,len(list_submit)):
  df_ = pd.read_feather(f'{LOCAL_DATA_PATH}{list_submit[i]}')
  df_.loc[df_['cpe_model_name'].isna() ,'cpe_model_name'] = '0'
  df = pd.concat([df, df_], ignore_index=True)
  #контролль цикла
  print (list_submit[i],'shape', df_.shape)
  #print ('Суммарное df.shape',df.shape)


# сохранение 
#df.to_pickle(f'{LOCAL_DATA_PATH}/submit_df_all_parts.pkl')
df.to_feather(f'{LOCAL_DATA_PATH}/submit_df_all_parts.feather')
#df.to_csv(f'{LOCAL_DATA_PATH}/submit_df_all_parts.csv')
print ('Итоговый файл сохранен на диск')

submit_part_0.feather shape (14368, 2506)
submit_part_1.feather shape (14463, 2506)
submit_part_2.feather shape (14366, 2505)
submit_part_3.feather shape (14572, 2506)
submit_part_4.feather shape (14315, 2505)
submit_part_5.feather shape (14426, 2506)
submit_part_6.feather shape (14485, 2506)
submit_part_7.feather shape (14616, 2506)
submit_part_8.feather shape (14623, 2505)
submit_part_9.feather shape (14490, 2506)
Итоговый файл сохранен на диск


In [ ]:
list_train_feat_age  = [
    'train_feat_age_part_0.feather',
    'train_feat_age_part_1.feather',
    'train_feat_age_part_2.feather',
    'train_feat_age_part_3.feather',
    'train_feat_age_part_4.feather',
    'train_feat_age_part_5.feather',
    'train_feat_age_part_6.feather',
    'train_feat_age_part_7.feather',
    'train_feat_age_part_8.feather',
    'train_feat_age_part_9.feather',
     ]

list_train_target_age  = [
    'train_target_age_part_0.csv',
    'train_target_age_part_1.csv',
    'train_target_age_part_2.csv',
    'train_target_age_part_3.csv',
    'train_target_age_part_4.csv',
    'train_target_age_part_5.csv',
    'train_target_age_part_6.csv',
    'train_target_age_part_7.csv',
    'train_target_age_part_8.csv',
    'train_target_age_part_9.csv',
     ]
list_train_feat_is_male  = [
    'train_feat_is_male_part_0.feather',
    'train_feat_is_male_part_1.feather',
    'train_feat_is_male_part_2.feather',
    'train_feat_is_male_part_3.feather',
    'train_feat_is_male_part_4.feather',
    'train_feat_is_male_part_5.feather',
    'train_feat_is_male_part_6.feather',
    'train_feat_is_male_part_7.feather',
    'train_feat_is_male_part_8.feather',
    'train_feat_is_male_part_9.feather',
     ]

list_train_target_is_male  = [
    'train_target_is_male_part_0.csv',
    'train_target_is_male_part_1.csv',
    'train_target_is_male_part_2.csv',
    'train_target_is_male_part_3.csv',
    'train_target_is_male_part_4.csv',
    'train_target_is_male_part_5.csv',
    'train_target_is_male_part_6.csv',
    'train_target_is_male_part_7.csv',
    'train_target_is_male_part_8.csv',
    'train_target_is_male_part_9.csv',
     ]

In [ ]:
list_train_feat_age  = [
    'train_feat_age_part_0.csv',
    'train_feat_age_part_1.csv',
    'train_feat_age_part_2.csv',
    'train_feat_age_part_3.csv',
    'train_feat_age_part_4.csv',
    'train_feat_age_part_5.csv',
    'train_feat_age_part_6.csv',
    'train_feat_age_part_7.csv',
    'train_feat_age_part_8.csv',
    'train_feat_age_part_9.csv',
     ]

list_train_target_age  = [
    'train_target_age_part_0.csv',
    'train_target_age_part_1.csv',
    'train_target_age_part_2.csv',
    'train_target_age_part_3.csv',
    'train_target_age_part_4.csv',
    'train_target_age_part_5.csv',
    'train_target_age_part_6.csv',
    'train_target_age_part_7.csv',
    'train_target_age_part_8.csv',
    'train_target_age_part_9.csv',
     ]
list_train_feat_is_male  = [
    'train_feat_is_male_part_0.feather',
    'train_feat_is_male_part_1.feather',
    'train_feat_is_male_part_2.feather',
    'train_feat_is_male_part_3.feather',
    'train_feat_is_male_part_4.feather',
    'train_feat_is_male_part_5.feather',
    'train_feat_is_male_part_6.feather',
    'train_feat_is_male_part_7.feather',
    'train_feat_is_male_part_8.feather',
    'train_feat_is_male_part_9.feather',
     ]

list_train_target_is_male  = [
    'train_target_is_male_part_0.csv',
    'train_target_is_male_part_1.csv',
    'train_target_is_male_part_2.csv',
    'train_target_is_male_part_3.csv',
    'train_target_is_male_part_4.csv',
    'train_target_is_male_part_5.csv',
    'train_target_is_male_part_6.csv',
    'train_target_is_male_part_7.csv',
    'train_target_is_male_part_8.csv',
    'train_target_is_male_part_9.csv',
     ]

### цикл склейки age

In [ ]:
%%time
#цикл склейки train_feat_age
#первый файл
df = pd.read_feather(f'{LOCAL_DATA_PATH}{list_train_feat_age[0]}')
#df = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_feat_age[0]}')
print (list_train_feat_age[0],'shape', df.shape)


for i in range (1,10):
  df_ = pd.read_feather(f'{LOCAL_DATA_PATH}{list_train_feat_age[i]}')
  #df_ = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_feat_age[i]}')   
  #сшиваем
  df = pd.concat([df, df_], ignore_index=True)
  #контролль цикла
  print (list_train_feat_age[i],'shape', df_.shape)
  #print ('Суммарное df.shape',df.shape)


# сохранение
df.to_feather(f'{LOCAL_DATA_PATH}/train_feat_age_df_all_parts.feather')
#df.to_csv(f'{LOCAL_DATA_PATH}/train_feat_age_df_all_parts.csv' ,index=False)

print ('Итоговый файл сохранен на диск')



#цикл склейки train_target_age
#первый файл
df = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_target_age[0]}')
print (list_train_target_age[0],'shape', df.shape)


for i in range (1,10):
  #df_ = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_target_age[i]}')
       
  #сшиваем
  df = pd.concat([df, pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_target_age[i]}')], ignore_index=True)
  #контролль цикла
  print (list_train_target_age[i],'shape', df_.shape)
  #print ('Суммарное df.shape',df.shape)


# сохранение
# pickle быстрее сохраняет( раз в 5-6) и читает, но  больше размером на диске в 6-7  раз
df.to_csv(f'{LOCAL_DATA_PATH}/train_target_age_df_all_parts.csv', index=False)
#df.to_pickle(f'{LOCAL_DATA_PATH}/train_target_age_df_all_parts.pkl')
#df.to_feather(f'{LOCAL_DATA_PATH}/train_target_age_df_all_parts.feather')

print ('Итоговый файл  train_target_age_df_all_parts сохранен на диск')


train_feat_age_part_0.feather shape (27081, 2505)
train_feat_age_part_1.feather shape (26936, 2505)
train_feat_age_part_2.feather shape (26801, 2504)
train_feat_age_part_3.feather shape (26901, 2505)
train_feat_age_part_4.feather shape (26794, 2504)
train_feat_age_part_5.feather shape (26962, 2505)
train_feat_age_part_6.feather shape (26817, 2505)
train_feat_age_part_7.feather shape (26839, 2505)
train_feat_age_part_8.feather shape (26883, 2504)
train_feat_age_part_9.feather shape (26908, 2505)
Итоговый файл сохранен на диск
train_target_age_part_0.csv shape (27081, 1)
train_target_age_part_1.csv shape (26908, 2505)
train_target_age_part_2.csv shape (26908, 2505)
train_target_age_part_3.csv shape (26908, 2505)
train_target_age_part_4.csv shape (26908, 2505)
train_target_age_part_5.csv shape (26908, 2505)
train_target_age_part_6.csv shape (26908, 2505)
train_target_age_part_7.csv shape (26908, 2505)
train_target_age_part_8.csv shape (26908, 2505)
train_target_age_part_9.csv shape (26908

### цикл склейки is_male

In [ ]:
#цикл склейки train_feat_is_male
#первый файл
df = pd.read_feather(f'{LOCAL_DATA_PATH}{list_train_feat_is_male[0]}')
print (list_train_feat_is_male[0],'shape', df.shape)


for i in range (1,len(list_train_feat_is_male)):
  df_ = pd.read_feather(f'{LOCAL_DATA_PATH}{list_train_feat_is_male[i]}')
       
  #сшиваем
  df = pd.concat([df, df_], ignore_index=True)
  #контролль цикла
  print (list_train_feat_is_male[i],'shape', df_.shape)
  #print ('Суммарное df.shape',df.shape)


# сохранение

df.to_feather(f'{LOCAL_DATA_PATH}/train_feat_is_male_df_all_parts.feather')
print ('Итоговый файл сохранен на диск')



#цикл склейки train_target_is_male
#первый файл
df = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_target_is_male[0]}')
print (list_train_target_is_male[0],'shape', df.shape)


for i in range (1,len(list_train_target_is_male)):
  df_ = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_target_is_male[i]}')
       
  #сшиваем
  df = pd.concat([df, df_], ignore_index=True)
  #контролль цикла
  print (list_train_target_is_male[i],'shape', df_.shape)
  #print ('Суммарное df.shape',df.shape)


# сохранение

df.to_csv(f'{LOCAL_DATA_PATH}/train_target_is_male_df_all_parts.csv', index=False)

#df.to_feather(f'{LOCAL_DATA_PATH}/train_target_is_male_df_all_parts.feather')

print ('Итоговый файл  train_target_is_male_df_all_parts сохранен на диск')

FileNotFoundError: ignored

In [ ]:

#цикл склейки train_target_is_male
#первый файл
df = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_target_is_male[0]}')
print (list_train_target_is_male[0],'shape', df.shape)

for i in range (1,len(list_train_target_is_male)):
  df_ = pd.read_csv(f'{LOCAL_DATA_PATH}{list_train_target_is_male[i]}')
       
  #сшиваем
  df = pd.concat([df, df_], ignore_index=True)
  #контролль цикла
  print (list_train_target_is_male[i],'shape', df_.shape)
  #print ('Суммарное df.shape',df.shape)


# сохранение

df.to_csv(f'{LOCAL_DATA_PATH}/train_target_is_male_df_all_parts.csv', index=False)

#df.to_feather(f'{LOCAL_DATA_PATH}/train_target_is_male_df_all_parts.feather')

print ('Итоговый файл  train_target_is_male_df_all_parts сохранен на диск')

train_target_is_male_part_0.csv shape (26529, 1)
train_target_is_male_part_1.csv shape (26394, 1)
train_target_is_male_part_2.csv shape (26269, 1)
train_target_is_male_part_3.csv shape (26338, 1)
train_target_is_male_part_4.csv shape (26251, 1)
train_target_is_male_part_5.csv shape (26410, 1)
train_target_is_male_part_6.csv shape (26287, 1)
train_target_is_male_part_7.csv shape (26356, 1)
train_target_is_male_part_8.csv shape (26316, 1)
train_target_is_male_part_9.csv shape (26358, 1)
Итоговый файл  train_target_is_male_df_all_parts сохранен на диск


In [ ]:
df.shape

(410594, 211)

# tmp


### cpe_manufacturer_name


In [ ]:
#загрузим данные
df_train = pd.read_pickle(f'{LOCAL_DATA_PATH}train_part_0.pkl')#.sample(frac=0.1, random_state=777)
df_train.shape


(27081, 20)

In [ ]:
df_train.head(1)

,user_id,day,evening,morning,night,cpe_model_name,cpe_manufacturer_name,apple.com,google.com,icloud.com,instagram.com,mail.yandex.ru,o2.mail.ru,online.sberbank.ru,play.google.com,vk.com,yandex.ru,zen.yandex.ru,age,is_male
0,4,199.0,170.0,212.0,10.0,Nova 3,Huawei,NaN,28.0,NaN,1.0,NaN,17.0,8.0,1.0,32.0,21.0,2.0,2,0


In [ ]:
df_train['pusto'] = ' '

In [ ]:
df_train['model'] = df_train['cpe_manufacturer_name'] + ' ' + df_train['cpe_model_name']

In [ ]:
df_train['model'].unique()

array(['Huawei Nova 3', 'Samsung Galaxy A10 Dual',
       'Samsung Galaxy A51 Dual', 'Samsung Galaxy A7 2017 Dual',
       'Apple iPhone 8 Plus', 'Apple iPhone XR', 'Huawei Honor 9 Lite',
       'Samsung Galaxy J6 Plus Dual', 'Huawei Honor 8A',
       'Huawei Honor 8X', 'Xiaomi Redmi Note 8T', 'Huawei Honor 9 Dual',
       'Samsung Galaxy A52', 'Huawei Honor 10 Lite', 'Apple iPhone 11',
       'Apple iPhone 5S', 'Samsung Galaxy A5 2017 Dual', 'Apple iPhone X',
       'Apple iPhone 11 Pro Max', 'Apple iPhone 6', 'Apple iPhone 7',
       'Apple iPhone 7 Plus', 'Huawei Honor 7A Pro',
       'Samsung Galaxy A50 Dual', 'Samsung Galaxy A9 2018 Dual',
       'Xiaomi Redmi 9C NFC', 'Apple iPhone 6S', 'Apple iPhone 11 Pro',
       'Samsung Galaxy A5 2016 Duos', 'Xiaomi Redmi Note 8 Pro',
       'Samsung Galaxy A30 Dual', 'Apple iPhone SE', 'Apple iPhone 8',
       'Huawei Honor 20', 'Apple iPhone Xs', 'Xiaomi Redmi 6A',
       'Huawei Honor 7S Dual', 'Samsung Galaxy M31',
       'Samsung Galaxy

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26529 entries, 0 to 27080
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   user_id                26529 non-null  int64  
 1   day                    25792 non-null  float64
 2   evening                24953 non-null  float64
 3   morning                25250 non-null  float64
 4   night                  21685 non-null  float64
 5   cpe_model_name         26118 non-null  object 
 6   cpe_manufacturer_name  26118 non-null  object 
 7   apple.com              9120 non-null   float64
 8   google.com             17817 non-null  float64
 9   icloud.com             6228 non-null   float64
 10  instagram.com          11166 non-null  float64
 11  mail.yandex.ru         15094 non-null  float64
 12  o2.mail.ru             10110 non-null  float64
 13  online.sberbank.ru     20273 non-null  float64
 14  play.google.com        16650 non-null  float64
 15  vk

## Посмотрим  по сколько строк на кажлм пльзователе.   
чтобы выкинуть тех у кого мало и котрые не дадут инфы для МЛ.    
тут я получил, что  чем больше возраст тем меньше на user_id строк с урлами, меньше кол-во среднего

In [ ]:
df.pivot_table(index='user_id', values='request_cnt', aggfunc=('count')).reset_index()

,user_id,request_cnt
0,4,591
1,16,1231
2,18,1125
3,26,1018
4,29,1458
...,...,...
27173,415227,4
27174,415251,3
27175,415266,3
27176,415273,1


,user_id,request_cnt
0,4,591
1,16,1231
2,18,1125


In [ ]:
df_pivot['request_cnt'].describe()

count    27178.000000
mean       786.966517
std       1045.336455
min          1.000000
25%        106.000000
50%        426.000000
75%       1060.000000
max      17481.000000
Name: request_cnt, dtype: float64

##Нужен список топ УРЛОВ

In [ ]:
df = pq.read_table(f'{LOCAL_DATA_PATH}part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet').\
select(['url_host','request_cnt']).to_pandas()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32638709 entries, 0 to 32638708
Data columns (total 2 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   url_host     object
 1   request_cnt  int64 
dtypes: int64(1), object(1)
memory usage: 498.0+ MB


In [ ]:
#всего урлов
# цикл грузит из паркета ид
df = pq.read_table(f'{LOCAL_DATA_PATH}{part_list[0]}').select(['url_host']).to_pandas()
for i in  range(3,7):
  df_ = pq.read_table(f'{LOCAL_DATA_PATH}{part_list[i]}').select(['url_host','request_cnt']).to_pandas()
  #соединяем
  df = pd.concat([df, df_], ignore_index=True)
  #контролль цикла
  print ('df_,shape',df_.shape)
  print ('df,shape',df.shape)


df_,shape (32156858, 2)
df,shape (64795567, 2)
df_,shape (32119163, 2)
df,shape (96914730, 2)
df_,shape (32300908, 2)
df,shape (129215638, 2)
df_,shape (32134423, 2)
df,shape (161350061, 2)


In [ ]:
df_urls = df['url_host'].value_counts().reset_index()#['url_host'].describe()

In [ ]:
df_urls

,index,url_host
0,googleads.g.doubleclick.net,10974508
1,yandex.ru,9493377
2,i.ytimg.com,8416659
3,vk.com,8319908
4,avatars.mds.yandex.net,8093162
...,...,...
154844,emelind.livejournal.com,1
154845,temhuk-livejournal-com.turbopages.org,1
154846,bbp.ru,1
154847,vestiukraine-livejournal-com.turbopages.org,1


In [ ]:
#в список топ
top_urls = set(df['url_host'].value_counts().head(2000).index)

In [ ]:
top_urls

Index(['googleads.g.doubleclick.net', 'yandex.ru', 'i.ytimg.com', 'vk.com',
       'avatars.mds.yandex.net', 'ad.mail.ru', 'yastatic.net', 'apple.com',
       'ads.adfox.ru', 'tpc.googlesyndication.com',
       ...
       'amic-ru.turbopages.org', 'qrcoder.ru', 'kino.rambler.ru',
       'service-online.su', 'friday.ru', 'tgramsearch.com',
       'bashinform-ru.turbopages.org', 'kogda-vykhodit.ru', 'snipp.ru',
       'rutaxist.ru'],
      dtype='object', length=2000)

In [ ]:
#cохраняем список топ урлов, чтобы тимур пометил не нужные
df_urls.to_csv(f'{LOCAL_DATA_PATH}/top_all_urls.csv', index=False)

In [ ]:
df_top_urls.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9636142 entries, 0 to 21388167
Data columns (total 14 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   region_name            object 
 1   city_name              object 
 2   cpe_manufacturer_name  object 
 3   cpe_model_name         object 
 4   url_host               object 
 5   cpe_type_cd            object 
 6   cpe_model_os_type      object 
 7   price                  float64
 8   date                   object 
 9   part_of_day            object 
 10  request_cnt            int64  
 11  user_id                int64  
 12  age                    float64
 13  is_male                object 
dtypes: float64(2), int64(2), object(10)
memory usage: 1.1+ GB


## FeaturesEng  \ Определяем нужны ли фичи
Загрузим df с таргетами ,все кроме урлов. Урежем таблицу. Зупустим ML.


In [ ]:
df.drop(columns=['url_host','date', 'region_name','city_name', 'price','request_cnt','part_of_day','cpe_type_cd' ], inplace=True)  #'part_of_day',

In [ ]:
df.drop(columns=['part_of_day'], inplace=True) #'url_host',

In [ ]:
#df.drop(columns=['cpe_type_cd','cpe_model_name'], inplace=True)

In [ ]:
#после удаления урлов у нас куча строк дубликатов
df.drop_duplicates(inplace=True)

In [ ]:
#выкидывыаем возраст меньше 19
df = df[df['age'] >= 19]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27081 entries, 0 to 21385071
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   cpe_model_name  27081 non-null  object 
 1   user_id         27081 non-null  int64  
 2   age             27081 non-null  float64
 3   is_male         27081 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.0+ MB


In [ ]:
df.head(2)

,cpe_model_name,user_id,age,is_male
0,Galaxy S20+,79395,35.0,1
1635,Poco X3 Pro,91294,35.0,1


In [ ]:
df.head(2)

,cpe_model_name,user_id,age,is_male,day,evening,morning,night
0,Galaxy S20+,79395,1,1,586.0,524.0,346.0,179.0
1,Poco X3 Pro,91294,1,1,551.0,515.0,338.0,166.0


# 3.ML

In [ ]:
#сабмиты и таргеты
id_for_target = pq.read_table(f'{LOCAL_DATA_PATH}/{TARGET_FILE}').to_pandas().query ('age >=19 ')
id_for_submit = pq.read_table(f'{LOCAL_DATA_PATH}/{SUBMISSION_FILE}').to_pandas()

In [ ]:
id_for_submit['user_id'].nunique()

144724

In [ ]:
id_for_target['user_id'].nunique()

268922

### Подгружаем файл с дииска
Трайн и сабмиты отдельно.   
Для age и is_male  отдельно.

In [ ]:
#df_train = pd.read_pickle(f'{LOCAL_DATA_PATH}/train_df_all_parts.pkl')
#df_train = pd.read_csv(f'{LOCAL_DATA_PATH}/train_df_all_parts.csv')   # csv  не подгруждается, не хватает памяти..


In [19]:
df_submit = pd.read_feather(f'{LOCAL_DATA_PATH}/submit_df_all_parts.feather')
df_submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144724 entries, 0 to 144723
Columns: 2506 entries, user_id to zyf03k.xyz
dtypes: float64(2504), int64(1), object(1)
memory usage: 2.7+ GB


In [17]:
df_submit.loc[df_submit['cpe_model_name'].isna() ,'cpe_model_name'] = '0'

In [18]:
df_submit.loc[df_submit['cpe_model_name'].isna() ,'cpe_model_name']

Series([], Name: cpe_model_name, dtype: object)

In [7]:
df_submit = df_submit.drop(['user_id'], axis=1)

0         635.0
1         132.0
2         704.0
3          82.0
4         189.0
          ...  
144719      NaN
144720      NaN
144721      NaN
144722      4.0
144723      NaN
Name: morning, Length: 144724, dtype: float64

In [7]:
# проверим размерности
#print ('df_train.shape =', df_train.shape)
print ('df_submit.shape =', df_submit.shape)

df_submit.shape = (144724, 2506)


## ML age

In [ ]:
# загружаем файл для age
#df_train_feat_age = pd.read_feather(f'{LOCAL_DATA_PATH}train_feat_age_df_all_parts.feather')
#df_train_target_age = pd.read_csv(f'{LOCAL_DATA_PATH}train_target_age_df_all_parts.csv')



In [6]:
#cat_features = ['cpe_manufacturer_name',	'cpe_model_os_type' ] #	,
cat_features = ['cpe_model_name'] 


### Поиск гиперпаратметров кэт для age,  use optuna
возьмем выборку 10% с общего файла или кусок один..


In [ ]:
#пока тут
import optuna
from sklearn.metrics import f1_score

In [ ]:
#загрузим данные
df_train = pd.read_feather(f'{LOCAL_DATA_PATH}train_part_0.feather')#.sample(frac=0.1, random_state=777)
df_train.shape

(27078, 2508)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(\
    df_train.drop(['user_id', 'age', 'is_male'], axis = 1), \
    df_train['age'], test_size = 0.33, random_state = 777)



In [ ]:
def objective(trial):
    
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_int('random_strength', 1, 100),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100.00)
    }

    model = CatBoostClassifier(**params, cat_features=cat_features, silent=True)
    model.fit(x_train, y_train, eval_set=(x_test, y_test), early_stopping_rounds=100, verbose=False)

    y_pred = model.predict(x_test)
    f1 = f1_score(y_test, y_pred, average='weighted')

    return 1.0 - f1

In [ ]:
%%time
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2023-03-01 17:52:45,921] A new study created in memory with name: no-name-b3d9549c-45e2-41e9-85a9-fe1107eea898
<ipython-input-111-1a69750e193a>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
<ipython-input-111-1a69750e193a>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100.00)
[I 2023-03-01 18:24:36,292] Trial 0 finished with value: 0.8364013876915181 and parameters: {'iterations': 309, 'learning_rate': 0.010184915467627361, 'depth': 10, 'l2_leaf_reg': 3, 'border_count': 255, 'random_stren

CatBoostError: ignored

In [ ]:
best_params = study.best_trial.params
best_params

{'iterations': 309,
 'learning_rate': 0.010184915467627361,
 'depth': 10,
 'l2_leaf_reg': 3,
 'border_count': 255,
 'random_strength': 53,
 'bagging_temperature': 1.1997682470432331}

In [ ]:
#это что отыскала оптуна до сбоя но получилась чушь , неа дефолтах в гораздо лучгше
# нужно переобучить оптуну, разхобраться что за х
best_params = {'iterations': 309,
 'learning_rate': 0.010184915467627361,
 'depth': 10,
 'l2_leaf_reg': 3,
 'border_count': 255,
 'random_strength': 53,
 'bagging_temperature': 1.1997682470432331}

In [ ]:
#best_params = study.best_trial.params
model = CatBoostClassifier(**best_params,cat_features=cat_features, silent=True )
model.fit(x_train, y_train)
print(metrics.classification_report(y_test, model.predict(x_test), \
    target_names = [ '19-25','26-34', '36-45', '46-56', '56-65', '65+']))

NameError: ignored

In [ ]:
print(metrics.classification_report(y_test, model.predict(x_test), \
    target_names = [ '19-25','26-34', '36-45', '46-56', '56-65', '65+']))

              precision    recall  f1-score   support

       19-25       0.00      0.00      0.00      1110
       26-34       0.33      1.00      0.49      2901
       36-45       0.28      0.01      0.01      2546
       46-56       0.00      0.00      0.00      1423
       56-65       0.00      0.00      0.00       776
         65+       0.00      0.00      0.00       180

    accuracy                           0.33      8936
   macro avg       0.10      0.17      0.08      8936
weighted avg       0.19      0.33      0.16      8936



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###Трайн,Тест Сплит

In [ ]:
#загрузим данные
df_train = pd.read_pickle(f'{LOCAL_DATA_PATH}train_part_0.pkl')#.sample(frac=0.1, random_state=777)
df_train.shape


(27081, 3341)

In [ ]:
df_train.columns

Index(['user_id', 'day', 'evening', 'morning', 'night', 'cpe_model_name', '-1',
       '0-hi--tech-mail-ru-0.cdn.ampproject.org', '03online.com', '1',
       ...
       'zooportal.pro', 'zr-ru.turbopages.org', 'zvonili.com', 'zvuch.com',
       'zvuk.com', 'zvukipro.com', 'zycrypto.com', 'zyf03k.xyz', 'age',
       'is_male'],
      dtype='object', length=3008)

In [ ]:
df_train['model'] = df_train['cpe_manufacturer_name'] + ' ' + df_train['cpe_model_name']
df_train = df_train.drop(['cpe_model_name','cpe_manufacturer_name'], axis=1)

In [ ]:
df_train = df_train.drop(['cpe_manufacturer_name'], axis=1)

In [ ]:
cat_features = ['cpe_model_name'] 
#cat_features = ['cpe_manufacturer_name']
#cat_features = ['model']

In [ ]:
cat_features

['cpe_model_name']

In [ ]:
df_train.loc[df_train['cpe_model_name'].isna() ,'cpe_model_name'] = 0

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(\
    df_train.drop(['user_id', 'age', 'is_male'], axis = 1), \
    df_train['age'], test_size = 0.33, random_state = 777)


In [ ]:
%%time
#тест модели 
clf = CatBoostClassifier()
clf.fit(x_train, y_train,cat_features, verbose = False)
print(metrics.classification_report(y_test, clf.predict(x_test), \
    target_names = [ '19-25','26-34', '36-45', '46-56', '56-65', '65+']))

              precision    recall  f1-score   support

       19-25       0.53      0.25      0.34      1099
       26-34       0.47      0.66      0.55      2924
       36-45       0.39      0.49      0.43      2575
       46-56       0.37      0.18      0.25      1430
       56-65       0.43      0.18      0.25       721
         65+       0.33      0.01      0.02       188

    accuracy                           0.43      8937
   macro avg       0.42      0.29      0.31      8937
weighted avg       0.43      0.43      0.40      8937

CPU times: user 17min 9s, sys: 1min 7s, total: 18min 16s
Wall time: 10min 55s


### Обучение на всей трайн выборке для сабмитов

In [ ]:
# подгружаем
df_train_feat_age = pd.read_feather(f'{LOCAL_DATA_PATH}train_feat_age_df_all_parts.feather')
df_train_target_age = pd.read_csv(f'{LOCAL_DATA_PATH}train_target_age_df_all_parts.csv')

In [ ]:
cat_features = ['cpe_model_name'] 

In [ ]:
%%time
# финальное обучение для сабмита
clf = CatBoostClassifier()
clf.fit(df_train_feat_age, df_train_target_age, cat_features, verbose = False)



CPU times: user 4h 34min 9s, sys: 1min 43s, total: 4h 35min 52s
Wall time: 2h 35min 55s


In [ ]:
##########################
# SAVE-LOAD using joblib #
##########################
# save
joblib.dump(clf, f'{LOCAL_DATA_PATH}model_age_v2.pkl')

['/content/drive/MyDrive/DS/MTS/model_age_v2.pkl']

In [8]:
# load
clf_age = joblib.load(f'{LOCAL_DATA_PATH}model_age_v2.pkl')

In [9]:
# предикты возраста
predict_age = clf_age.predict(df_submit)

In [ ]:
#сохраняем в файл
pd.Series(predict_age.reshape(-1)).to_pickle(f'{LOCAL_DATA_PATH}predict_age.pkl')


## ML is_male


In [ ]:
#загружаем данные для is_male
#df_train_feat_is_male = pd.read_pickle(f'{LOCAL_DATA_PATH}train_feat_is_male_df_all_parts.pkl')
#df_train_target_is_male = pd.read_pickle(f'{LOCAL_DATA_PATH}train_target_is_male_df_all_parts.pkl')
df_train_feat_is_male = pd.read_feather(f'{LOCAL_DATA_PATH}train_feat_is_male_df_all_parts.feather')
df_train_target_is_male = pd.read_feather(f'{LOCAL_DATA_PATH}train_target_is_male_df_all_parts.feather')

### Трайн Тест Сплит

In [ ]:
df_train_target_is_male['is_male'].unique()
df_train_target_is_male.shape

NameError: ignored

In [ ]:
cat_features = ['cpe_model_name'] 

In [ ]:
df_train['is_male'].unique()

array(['0', '1'], dtype=object)

In [ ]:
df_train = df_train[df_train['is_male'] != 'NA']

In [ ]:
%%time
x_train, x_test, y_train, y_test = train_test_split(\
    df_train.drop(['user_id', 'age', 'is_male'], axis = 1), df_train['is_male'], test_size = 0.33, random_state = 777)
clf = CatBoostClassifier()
clf.fit(x_train, y_train, cat_features , verbose = False )


CPU times: user 3min 37s, sys: 5.23 s, total: 3min 43s
Wall time: 2min 15s


In [ ]:
print(f'GINI по полу {2 * metrics.roc_auc_score(y_test, clf.predict_proba(x_test)[:,1]) - 1:2.3f}')

GINI по полу 0.667


### Обучение на всей выбоке для сабмитов

In [ ]:
df_train_feat_is_male = pd.read_feather(f'{LOCAL_DATA_PATH}train_feat_is_male_df_all_parts.feather')
df_train_target_is_male = pd.read_csv(f'{LOCAL_DATA_PATH}train_target_is_male_df_all_parts.csv')

In [ ]:
cat_features = ['cpe_model_name'] 

In [ ]:
df_train_target_is_male.head()


,is_male
0,0
1,0
2,1
3,0
4,0


In [ ]:
%%time

# обучения для сабмита на всей выборке
clf = CatBoostClassifier()
clf.fit(df_train_feat_is_male, df_train_target_is_male, cat_features , verbose = False )


CPU times: user 43min 28s, sys: 8.37 s, total: 43min 36s
Wall time: 24min 39s


In [ ]:
##########################
# SAVE-LOAD using joblib #
##########################
joblib.dump(clf, f'{LOCAL_DATA_PATH}model_is_male_v2.pkl')

['/content/drive/MyDrive/DS/MTS/model_is_male_v2.pkl']

In [10]:
# load
clf_is_male = joblib.load(f'{LOCAL_DATA_PATH}model_is_male_v2.pkl')

In [11]:
predict_is_male_proba = clf_is_male.predict_proba(df_submit)

In [ ]:
#сохранем
pd.Series(predict_is_male_proba[:,1]).to_pickle(f'{LOCAL_DATA_PATH}predict_is_male_proba.pkl')

In [14]:
predict_is_male_proba[:,0].shape

(144724,)

## Заполняем id_to_submit

In [ ]:
#меняем категорию
def fix_age_cat(age):
  if age == 0:
    return 1
  elif age == 1:
    return 2
  elif age == 2:
    return 3
  elif age == 3:
    return 4
  elif age == 4:
    return 5
  return 6

In [20]:
#это временный дата сет, т к у нас нет некоторого числа user_id  
a = pd.DataFrame (df_submit['user_id'])

In [21]:
a['is_male'] = predict_is_male_proba[:,1]
a['age'] =  predict_age

In [ ]:
# fix категории
a['age'] = a['age'].map(fix_age_cat)

In [22]:
a['age'].unique()

array([2, 1, 3, 4, 5, 6])

In [23]:
id_to_submit = pq.read_table(f'{LOCAL_DATA_PATH}/{SUBMISSION_FILE}').to_pandas().reset_index(drop=True)

In [15]:
id_to_submit.shape
#a.shape

(144724, 3)

In [14]:
# когда все user_id  в предиктах
id_to_submit['is_male'] = predict_is_male_proba[:,1]
id_to_submit['age'] =  predict_age

In [24]:
#этот мердж правльный
id_to_submit = id_to_submit.merge(a,how='outer', on='user_id')


In [25]:
id_to_submit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144724 entries, 0 to 144723
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  144724 non-null  int64  
 1   is_male  144724 non-null  float64
 2   age      144724 non-null  int64  
dtypes: float64(1), int64(2)
memory usage: 4.4 MB


In [ ]:
#заполним пропуски is_male = 0.511735 , age = 2 or 3

In [ ]:
#закрывем na
id_to_submit.loc[id_to_submit['age'].isna(),'age'] = 3
id_to_submit.loc[id_to_submit['is_male'].isna(),'is_male'] = 0.52
id_to_submit['age'] = id_to_submit['age'].astype(int)

In [26]:
id_to_submit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144724 entries, 0 to 144723
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  144724 non-null  int64  
 1   is_male  144724 non-null  float64
 2   age      144724 non-null  int64  
dtypes: float64(1), int64(2)
memory usage: 4.4 MB


In [27]:
id_to_submit.head()

,user_id,is_male,age
0,221301,0.961782,2
1,31271,0.451565,3
2,211594,0.523701,2
3,253119,0.494832,3
4,192578,0.811708,2


In [28]:
#итоговый сохраняем
id_to_submit.to_csv(f'{LOCAL_DATA_PATH}/submission_v4_1.csv', index = False)

## СКоры на лидер борде    
Когда пофиксил категоризацию age
	1,6359491017   это 2300 тпо урлов без отсева технических, кэт дефол